In [2]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=958f38d6f09cbb1e6b016744e2d3259ca885c3eeaa9508e25e3ffc8ab57e2226
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col, when, isnan, isnull, count, avg, trim
import os

In [4]:
DATA_FOLDER = "data"
# source https://www.statista.com/statistics/242030/marital-status-of-the-us-population-by-sex/
# the first value is male and the second is for female
MARITAL_STATUS_BY_GENDER = [
    ["Never-married", 47.35, 41.81],
    ["Married-AF-spouse", 67.54, 68.33],
    ["Widowed", 3.58, 11.61],
    ["Divorced", 10.82, 15.09]
]
MARITAL_STATUS_BY_GENDER_COLUMNS = ["marital_status_statistics", "male", "female"]

In [5]:




def read_data(spark: SparkSession) -> DataFrame:
    """
    read data based on the given schema; this is much faster than spark determining the schema
    """

    # Define the schema for the dataset
    schema = StructType([
        StructField("age", IntegerType(), True),
        StructField("workclass", StringType(), True),
        StructField("fnlwgt", FloatType(), True),
        StructField("education", StringType(), True),
        StructField("education_num", FloatType(), True),
        StructField("marital_status", StringType(), True),
        StructField("occupation", StringType(), True),
        StructField("relationship", StringType(), True),
        StructField("race", StringType(), True),
        StructField("sex", StringType(), True),
        StructField("capital_gain", FloatType(), True),
        StructField("capital_loss", FloatType(), True),
        StructField("hours_per_week", FloatType(), True),
        StructField("native_country", StringType(), True),
        StructField("income", StringType(), True)
    ])

    # Read the dataset
    data = spark.read \
        .schema(schema) \
        .option("header", "false") \
        .option("inferSchema", "false") \
        .csv(os.path.join(DATA_FOLDER,"*.csv"))

    data = data.repartition(8)

    float_columns = [f.name for f in data.schema.fields if isinstance(f.dataType, FloatType)]
    for v in float_columns:
        data = data.withColumn(v, data[v].cast(IntegerType()))

    # Get the names of all StringType columns
    string_columns = [f.name for f in data.schema.fields if isinstance(f.dataType, StringType)]

    # Remove leading and trailing spaces in all string columns
    for column in string_columns:
        data = data.withColumn(column, trim(data[column]))

    # Show the first 5 rows of the dataset
    data.show(5)

    return data


In [6]:

def missing_values(data: DataFrame) -> DataFrame:
    """
    count the number of samples with missing values for each row
    remove such samples
    """

    missing_values = data.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in data.columns])

    # Show the missing values count per column
    missing_values.show()

    # Get the number of samples in the DataFrame
    num_samples = data.count()

    # Print the number of samples
    print("Number of samples:", num_samples)

    data = data.dropna()

    return data



In [7]:
def feature_engineering(data: DataFrame) -> DataFrame:
    """
    calculate the product of each pair of integer features
    """

    # Create columns consisting of all products of columns of type IntegerType
    integer_columns = [f.name for f in data.schema.fields if isinstance(f.dataType, IntegerType)]
    for i, col1 in enumerate(integer_columns):
        for col2 in integer_columns[i:]:
            product_col_name = f"{col1}_x_{col2}"
            data = data.withColumn(product_col_name, col(col1) * col(col2))

    data.show(5)

    return data

def bias_marital_status(data: DataFrame):
    """
    is there bias in capital gain by marital status
    """

    # Calculate the average capital_gain by marital_status
    average_capital_gain = data.groupBy("marital_status").agg(avg("capital_gain").alias("average_capital_gain"))

    # Show the average capital_gain by marital_status
    average_capital_gain.show()

    # Filter data based on marital_status = Divorced
    divorced_data = data.filter(data.marital_status == "Divorced")

    # Show the first 5 rows of the filtered DataFrame
    divorced_data.show(5)

def join_with_US_gender(spark: SparkSession, data: DataFrame):
    """
    join with respect to the marital_status
    """

    # create a data frame from new data
    columns = ["dept_name","dept_id"]
    us_df = spark.createDataFrame(MARITAL_STATUS_BY_GENDER, MARITAL_STATUS_BY_GENDER_COLUMNS)

    return data.join(us_df, data.marital_status == us_df.marital_status_statistics, 'outer')

def main():
    # Create a Spark session
    spark = SparkSession.builder \
        .appName("Read Adult Dataset") \
        .getOrCreate()

    data = read_data(spark)
    # perform basic EDA - count missing values
    data = missing_values(data)
    data = feature_engineering(data)
    bias_marital_status(data)
    data = join_with_US_gender(spark, data)



    data.show(5)
    data.write.format('csv').option('header', 'true').mode('overwrite').save('saved.csv')
    #spark.stop()
    #return data


In [8]:
def bias_marital_status(data: DataFrame):
    """
    is there bias in capital gain by marital status
    """

    # Calculate the average capital_gain by marital_status
    average_capital_gain = data.groupBy("marital_status").agg(avg("capital_gain").alias("average_capital_gain"))

    # Show the average capital_gain by marital_status
    average_capital_gain.show()

    # Filter data based on marital_status = Divorced
    divorced_data = data.filter(data.marital_status == "Divorced")

    # Show the first 5 rows of the filtered DataFrame
    divorced_data.show(5)



In [9]:
def join_with_US_gender(spark: SparkSession, data: DataFrame):
    """
    join with respect to the marital_status
    """

    # create a data frame from new data
    columns = ["dept_name","dept_id"]
    us_df = spark.createDataFrame(MARITAL_STATUS_BY_GENDER, MARITAL_STATUS_BY_GENDER_COLUMNS)

    return data.join(us_df, data.marital_status == us_df.marital_status_statistics, 'outer')



In [ ]:
spark = SparkSession.builder \
        .appName("Read Adult Dataset") \
        .getOrCreate()

data = read_data(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/25 15:29:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+---------+------+------------+-------------+------------------+---------------+------------+------------------+------+------------+------------+--------------+--------------+------+
|age|workclass|fnlwgt|   education|education_num|    marital_status|     occupation|relationship|              race|   sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+---------+------+------------+-------------+------------------+---------------+------------+------------------+------+------------+------------+--------------+--------------+------+
| 45|  Private|191098|   Bachelors|           13|Married-civ-spouse| Prof-specialty|     Husband|Asian-Pac-Islander|  Male|           0|           0|            40|         China| <=50K|
| 46|  Private|250821| Prof-school|           15|          Divorced|Farming-fishing|   Unmarried|             White|  Male|           0|           0|            48| United-States| <=50K|
| 53|  Private|242859|Some-college|           10|         Separat

In [ ]:
data = read_data(spark)

+---+---------+------+------------+-------------+------------------+---------------+------------+------------------+------+------------+------------+--------------+--------------+------+
|age|workclass|fnlwgt|   education|education_num|    marital_status|     occupation|relationship|              race|   sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+---------+------+------------+-------------+------------------+---------------+------------+------------------+------+------------+------------+--------------+--------------+------+
| 45|  Private|191098|   Bachelors|           13|Married-civ-spouse| Prof-specialty|     Husband|Asian-Pac-Islander|  Male|           0|           0|            40|         China| <=50K|
| 46|  Private|250821| Prof-school|           15|          Divorced|Farming-fishing|   Unmarried|             White|  Male|           0|           0|            48| United-States| <=50K|
| 53|  Private|242859|Some-college|           10|         Separat

In [ ]:
data = missing_values(data)

+---+---------+------+---------+-------------+--------------+----------+------------+----+---+------------+------------+--------------+--------------+------+
|age|workclass|fnlwgt|education|education_num|marital_status|occupation|relationship|race|sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+---------+------+---------+-------------+--------------+----------+------------+----+---+------------+------------+--------------+--------------+------+
|  0|        0|     0|        0|            0|             0|         0|           0|   0|  0|           0|           0|             0|             0|     0|
+---+---------+------+---------+-------------+--------------+----------+------------+----+---+------------+------------+--------------+--------------+------+

Number of samples: 32561


In [ ]:
data = feature_engineering(data)

24/05/25 15:30:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+---------+------+------------+-------------+------------------+---------------+------------+------------------+------+------------+------------+--------------+--------------+------+---------+------------+-------------------+------------------+------------------+--------------------+---------------+----------------------+---------------------+---------------------+-----------------------+-----------------------------+----------------------------+----------------------------+------------------------------+---------------------------+---------------------------+-----------------------------+---------------------------+-----------------------------+-------------------------------+
|age|workclass|fnlwgt|   education|education_num|    marital_status|     occupation|relationship|              race|   sex|capital_gain|capital_loss|hours_per_week|native_country|income|age_x_age|age_x_fnlwgt|age_x_education_num|age_x_capital_gain|age_x_capital_loss|age_x_hours_per_week|fnlwgt_x_fnlwgt|fnlwg

In [ ]:
bias_marital_status(data)

+--------------------+--------------------+
|      marital_status|average_capital_gain|
+--------------------+--------------------+
|           Separated|   535.5687804878049|
|       Never-married|  376.58831788823363|
|Married-spouse-ab...|   653.9832535885167|
|            Divorced|   728.4148098131893|
|             Widowed|   571.0715005035247|
|   Married-AF-spouse|   432.6521739130435|
|  Married-civ-spouse|  1764.8595085470085|
+--------------------+--------------------+

+---+---------+------+------------+-------------+--------------+---------------+--------------+-----+------+------------+------------+--------------+--------------+------+---------+------------+-------------------+------------------+------------------+--------------------+---------------+----------------------+---------------------+---------------------+-----------------------+-----------------------------+----------------------------+----------------------------+------------------------------+----------------

In [ ]:
data = join_with_US_gender(spark, data)

In [ ]:
def main():
    # Create a Spark session
    spark = SparkSession.builder \
        .appName("Read Adult Dataset") \
        .getOrCreate()

    data = read_data(spark)
    # perform basic EDA - count missing values
    data = missing_values(data)
    data = feature_engineering(data)
    bias_marital_status(data)
    data = join_with_US_gender(spark, data)



    data.show(5)
    spark.stop()


In [12]:
from pyspark.sql import functions as F


try:
   # Select rows where age is between 30 and 50
   filtered_data = data.filter((F.col("age") >= 30) & (F.col("age") <= 50))


   # Check if the DataFrame is empty using a more efficient method
   if filtered_data.limit(1).count() > 0:
       # Compute summary statistics directly in Spark
       summary_stats = filtered_data.describe().show()


       # If you still need to convert to Pandas DataFrame for specific reasons, do it after filtering and aggregations
       pandas_df = filtered_data.toPandas()
       print(pandas_df.describe())
   else:
       print("No data found in the specified age range.")


except Exception as e:
   print("An error occurred:", e)



+-------+------------------+------------+------+
|summary|               age|   workclass|fnlwgt|
+-------+------------------+------------+------+
|  count|             16389|       16389|     0|
|   mean|39.298126792360726|        NULL|  NULL|
| stddev| 5.895184014315313|        NULL|  NULL|
|    min|                30|           ?|  NULL|
|    max|                50| Without-pay|  NULL|
+-------+------------------+------------+------+

                age  fnlwgt
count  16389.000000     0.0
mean      39.298127     NaN
std        5.895184     NaN
min       30.000000     NaN
25%       34.000000     NaN
50%       39.000000     NaN
75%       44.000000     NaN
max       50.000000     NaN
